In [16]:
import pandas as pd
import numpy as np
import random
import joblib
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df.head()

hemoglobin    ferritin  ret_count  segmented_neutrophils        tibc  \
0   11.183192  187.573466   4.951674               1.661383  316.593436   
1   14.387445         NaN        NaN                    NaN         NaN   
2   12.749357    5.012158   3.502800               6.179371  498.418768   
3   11.508870  197.180945   1.200125               0.000000  457.033309   
4    9.456656  427.952052        NaN               0.660252         NaN   

          mcv  serum_iron       rbc        age  gender  indirect_bilirubin  \
0   95.006424  181.242992  6.758433  60.586525       1            2.976104   
1         NaN  125.249617  6.350652  86.964793       1            2.968983   
2   76.759285  159.834784  5.018156  67.065069       1            2.705719   
3  102.900301  131.177927  6.656823  66.403181       1            1.869338   
4  104.543774         NaN       NaN        NaN       0                 NaN   

   transferrin  creatinine  cholestrol      copper    ethanol     folate  \
0   282.291951         NaN   28.040619   96.721542  49.530542  23.186628   
1   158.844879         NaN   98.357508  112.758764  62.464566   7.168920   
2   442.293823    1.022939   56.850479   75.739552  72.072041  20.600875   
3   223.896404         NaN  111.220307   66.999185  18.353272  14.132423   
4          NaN         NaN         NaN         NaN        NaN        NaN   

     glucose                                 label  
0        NaN                      Hemolytic anemia  
1        NaN                             No anemia  
2  44.872138                Iron deficiency anemia  
3        NaN                    Unspecified anemia  
4        NaN  Vitamin B12/Folate deficiency anemia

In [18]:
utils.get_dt_performance(df)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

(0.9997142857142857,
 0.999678677273867,
 0.9998185933055129,
 datetime.timedelta(microseconds=3997))

#### Adding correlated features

In [19]:
df['hematocrit'] = 3*df['hemoglobin']
df['rbc'] = (30*df['hemoglobin'])/df['mcv']
df['tsat'] = (df['serum_iron']/df['tibc'])*100
df.head()

hemoglobin    ferritin  ret_count  segmented_neutrophils        tibc  \
0   11.183192  187.573466   4.951674               1.661383  316.593436   
1   14.387445         NaN        NaN                    NaN         NaN   
2   12.749357    5.012158   3.502800               6.179371  498.418768   
3   11.508870  197.180945   1.200125               0.000000  457.033309   
4    9.456656  427.952052        NaN               0.660252         NaN   

          mcv  serum_iron       rbc        age  gender  ...  transferrin  \
0   95.006424  181.242992  3.531295  60.586525       1  ...   282.291951   
1         NaN  125.249617       NaN  86.964793       1  ...   158.844879   
2   76.759285  159.834784  4.982859  67.065069       1  ...   442.293823   
3  102.900301  131.177927  3.355346  66.403181       1  ...   223.896404   
4  104.543774         NaN  2.713693        NaN       0  ...          NaN   

   creatinine  cholestrol      copper    ethanol     folate    glucose  \
0         NaN   28.040619   96.721542  49.530542  23.186628        NaN   
1         NaN   98.357508  112.758764  62.464566   7.168920        NaN   
2    1.022939   56.850479   75.739552  72.072041  20.600875  44.872138   
3         NaN  111.220307   66.999185  18.353272  14.132423        NaN   
4         NaN         NaN         NaN        NaN        NaN        NaN   

                                  label hematocrit       tsat  
0                      Hemolytic anemia  33.549575  57.247868  
1                             No anemia  43.162335        NaN  
2                Iron deficiency anemia  38.248071  32.068372  
3                    Unspecified anemia  34.526609  28.702050  
4  Vitamin B12/Folate deficiency anemia  28.369968        NaN  

[5 rows x 21 columns]

#### Dropping some columns

In [20]:
df.columns

Index(['hemoglobin', 'ferritin', 'ret_count', 'segmented_neutrophils', 'tibc',
       'mcv', 'serum_iron', 'rbc', 'age', 'gender', 'indirect_bilirubin',
       'transferrin', 'creatinine', 'cholestrol', 'copper', 'ethanol',
       'folate', 'glucose', 'label', 'hematocrit', 'tsat'],
      dtype='object')

In [21]:
cols_to_drop = ['transferrin', 'age', 'indirect_bilirubin']
df = df.drop(cols_to_drop, axis=1)
df = df[[c for c in df if c != 'label'] + ['label']]
df.head()

hemoglobin    ferritin  ret_count  segmented_neutrophils        tibc  \
0   11.183192  187.573466   4.951674               1.661383  316.593436   
1   14.387445         NaN        NaN                    NaN         NaN   
2   12.749357    5.012158   3.502800               6.179371  498.418768   
3   11.508870  197.180945   1.200125               0.000000  457.033309   
4    9.456656  427.952052        NaN               0.660252         NaN   

          mcv  serum_iron       rbc  gender  creatinine  cholestrol  \
0   95.006424  181.242992  3.531295       1         NaN   28.040619   
1         NaN  125.249617       NaN       1         NaN   98.357508   
2   76.759285  159.834784  4.982859       1    1.022939   56.850479   
3  102.900301  131.177927  3.355346       1         NaN  111.220307   
4  104.543774         NaN  2.713693       0         NaN         NaN   

       copper    ethanol     folate    glucose  hematocrit       tsat  \
0   96.721542  49.530542  23.186628        NaN   33.549575  57.247868   
1  112.758764  62.464566   7.168920        NaN   43.162335        NaN   
2   75.739552  72.072041  20.600875  44.872138   38.248071  32.068372   
3   66.999185  18.353272  14.132423        NaN   34.526609  28.702050   
4         NaN        NaN        NaN        NaN   28.369968        NaN   

                                  label  
0                      Hemolytic anemia  
1                             No anemia  
2                Iron deficiency anemia  
3                    Unspecified anemia  
4  Vitamin B12/Folate deficiency anemia

#### Saving dataset 

In [23]:
df.to_csv('../../data/more_features/more_feats_correlated_0.1.csv', index=False)